In [1]:
import sys
sys.path.append('../edaf')

In [2]:
from plot_helpers import *
from data_helpers import *
from decomp import *
from sortedcontainers import SortedList, SortedDict
from loguru import logger
import os, sys, gzip, json
import sqlite3
import pandas as pd
from edaf.core.uplink.preprocess import preprocess_ul
from edaf.core.uplink.analyze_packet import ULPacketAnalyzer
from edaf.core.uplink.analyze_channel import ULChannelAnalyzer
from edaf.core.uplink.analyze_scheduling import ULSchedulingAnalyzer
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import JSON
import json


from pathlib import Path
import ijson

%load_ext autoreload
%autoreload 2

In [3]:
# Remove default handler
logger.remove()
# Add a new handler with level WARNING
logger.add(sys.stdout, level="ERROR")

16

In [5]:
MEAS_DIR = '../S1-meas/' ### change the paths here
DATA_DIR_PREFIX = './data/'

In [7]:
for i in [9]:
    MEAS_LABEL = f's{i}'
    DB_FILE = MEAS_DIR+MEAS_LABEL+'/'+'res_db.sql'
    DATA_DIR = DATA_DIR_PREFIX+MEAS_LABEL+'/'
    
    if not os.path.exists(DB_FILE):
        print(f'skipping, {DB_FILE} does not exist')
        continue
    print(f'processing {DB_FILE}')
    # packet analyzer
    analyzer = ULPacketAnalyzer(DB_FILE)
    uids_arr = range(analyzer.first_ueipid, analyzer.last_ueipid+1)
    packets = analyzer.figure_packettx_from_ueipids(uids_arr)
    
    # sched analyzer
    sched_analyzer = ULSchedulingAnalyzer(
        total_prbs_num = 106, 
        symbols_per_slot = 14, 
        slots_per_frame = 20, 
        slots_duration_ms = 0.5, 
        scheduling_map_num_integers = 4,
        max_num_frames=100,
        db_addr = DB_FILE
    )
    
    begin_ts = sched_analyzer.first_ts
    end_ts = sched_analyzer.last_ts
    sched_arr = sched_analyzer.find_resource_schedules_from_ts(begin_ts, end_ts)
    sched_sorted_dict = SortedDict({sched['decision_ts']: sched for sched in sched_arr})
    bsrupd_arr = sched_analyzer.find_bsr_upd_from_ts(begin_ts, end_ts)
    sr_tx_arr = sched_analyzer.find_sr_tx_from_ts(begin_ts, end_ts)
    bsr_tx_arr = sched_analyzer.find_bsr_tx_from_ts(begin_ts, end_ts)
    sr_bsr_tx_sorted_list = SortedList([sr_tx['timestamp'] for sr_tx in sr_tx_arr]+[bsr_tx['timestamp'] for bsr_tx in bsr_tx_arr])
    
    chan_analyzer = ULChannelAnalyzer(DB_FILE)
    begin_ts = chan_analyzer.first_ts
    end_ts = chan_analyzer.last_ts
    WINDOW_LEN_SECONDS = 2
    mcs_arr = chan_analyzer.find_mcs_from_ts(begin_ts,end_ts)
    tb_arr = chan_analyzer.find_mac_attempts_from_ts(begin_ts,end_ts)
    mcs_sorted_dict = SortedDict({e['timestamp']: e['mcs'] for e in mcs_arr})
    
    # Ensure the DATA_DIR exists
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
        
    # Call the function
    create_json_dump(packets, DATA_DIR+'packets.json')
    create_json_dump(sr_tx_arr, DATA_DIR+'sr_tx.json')
    create_json_dump(bsrupd_arr, DATA_DIR+'bsr_upd.json')
    create_json_dump(bsr_tx_arr, DATA_DIR+'bsr_tx.json')
    create_json_dump(sched_arr, DATA_DIR+'sched.json')
    create_json_dump(mcs_arr, DATA_DIR+'mcs.json')
    create_json_dump(tb_arr, DATA_DIR+'tb.json')


processing ../S1-meas/s9/res_db.sql
2024-11-01 12:24:49.092 | ERROR    | edaf.core.uplink.analyze_packet:figure_packettx_from_ueipids:118 - UE SN 8490 for UE IP ID 1513 could not be found on GNB side. Dropped packet?
2024-11-01 12:24:49.100 | ERROR    | edaf.core.uplink.analyze_packet:figure_packettx_from_ueipids:118 - UE SN 8491 for UE IP ID 1512 could not be found on GNB side. Dropped packet?
